1:1 비율 down sampling

FE2의 feature

In [1]:
import pandas as pd
from lightgbm import Booster
import lightgbm as lgb
import copy
import time
import json

In [2]:
train_path = '/home/workspace/user-workspace/slim_train.parquet'
test_path ='/home/workspace/user-workspace/slim_test.parquet'
encoder = '/home/workspace/user-workspace/cat_encoder.json'
decoder = '/home/workspace/user-workspace/inverse_cat_encoder.json'
data_dir = '/home/workspace/user-workspace/junheon/data/task150/'
model_dir = '/home/workspace/user-workspace/junheon/model/task150/'
result_dir = '/home/workspace/user-workspace/junheon/result/task150/'
submission_dir = '/home/workspace/user-workspace/prediction/'

In [3]:
bagging_size = 5
prefix = "ex12"

In [4]:
features = [
    "basic_feature",
    "month_day",
    "pay_amt",
    "PCA5",
    "NMF5",
    "age_bin",
    "phone_cnt_wrt_seq",
    "seq_count",
    "is_targeted"
]

In [5]:
categorical_features = [
    "COMMC_CLF", "NPAY_YN", "PAY_MTHD_CD", "ARS_AUTHTI_YN", "GNDR", "FOREI_YN",  "AUTHTI_CLF_FLG", 
    "SVC_CLF_NM", "CP_M_CLF_NM", "CP_S_CLF_NM", "month", "day", "AGE_bin"
]

In [6]:
remove_features = ["AGE"]

# merge data

### train

In [7]:
for seed in range(bagging_size):
    df_list = []
    for name in features:
        feature_df = pd.read_parquet(f"{data_dir}{name}_{seed}.parquet").set_index("id")
        df_list.append(feature_df)
    df = pd.concat(df_list, axis=1)
    df = df.drop(columns=remove_features, axis=1)
    df.to_parquet(f"{data_dir}{prefix}_train_{seed}.parquet")

### test

In [8]:
df_list = []
for name in features:
    feature_df = pd.read_parquet(f"{data_dir}{name}_test.parquet").set_index("id")
    df_list.append(feature_df)
df = pd.concat(df_list, axis=1)
df = df.drop(remove_features, axis=1)
df.to_parquet(f"{data_dir}{prefix}_test.parquet")

# Modeling

In [9]:
model_params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "num_leaves": 255,
    "max_depth": 8,
    "min_child_samples": 200,
    "subsample": 0.9,
    "subsample_freq": 1,
    "colsample_bytree": 0.5,
    "min_child_weight": 0,
    "subsample_for_bin": 1000000,
    "min_split_gain": 0,
    "reg_lambda": 0,
    "verbose": 0,
    "seed": 777
}

train_params = {
    "num_boost_round": 2000,
    "early_stopping_rounds": 30
}

### train and valid

In [10]:
boosters = []
train_results = []

In [ ]:
for seed in range(bagging_size):
    start_time = time.time()
    
    train_df = pd.read_parquet(f"{data_dir}{prefix}_train_{seed}.parquet")
    valid_df = train_df[train_df['month']==10]
    train_df = train_df[train_df['month']!=10]
    train_dataset = lgb.Dataset(train_df.loc[:, train_df.columns != "target"], label=train_df["target"].values)
    valid_dataset = lgb.Dataset(valid_df.loc[:, valid_df.columns != "target"], label=valid_df["target"].values)
    eval_results = {}
    
    model: Booster = lgb.train(model_params,
                              train_dataset,
                              categorical_feature=categorical_features,
                              valid_sets=[train_dataset, valid_dataset],
                              valid_names=['train', 'valid'],
                              evals_result=eval_results,
                              **train_params)
        
    best_iteration = model.best_iteration
    
    params = copy.deepcopy(train_params)
    params["num_boost_round"] = best_iteration
    if "early_stopping_rounds" in params:
        del params["early_stopping_rounds"]
        
    train_df = pd.read_parquet(f"{data_dir}{prefix}_train_{seed}.parquet")
    train_dataset = lgb.Dataset(train_df.loc[:, train_df.columns != "target"], label=train_df["target"].values)
    model = lgb.train(model_params,
                     train_dataset,
                     categorical_feature=categorical_features,
                     **params)
    
    model.save_model(f"{model_dir}{prefix}_model_{seed}.txt")
    boosters.append(model)
    
    result = dict()
    result['train_auc'] = eval_results['train']['auc'][best_iteration-1]
    result['valid_auc'] = eval_results['valid']['auc'][best_iteration-1]
    result['best_iteration'] = best_iteration
    result['train_time'] = time.time() - start_time
    result['feature_importance'] = {name: int(score) for name, score in zip(model.feature_name(), model.feature_importance())}
    result['feature_importance'] = {key: value for key, value in sorted(result['feature_importance'].items(), key=lambda item: item[1], reverse=True)}
    train_results.append(result)

/home/workspace/user-workspace/lib/lightgbm/basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['AGE_bin', 'ARS_AUTHTI_YN', 'AUTHTI_CLF_FLG', 'COMMC_CLF', 'CP_M_CLF_NM', 'CP_S_CLF_NM', 'FOREI_YN', 'GNDR', 'NPAY_YN', 'PAY_MTHD_CD', 'SVC_CLF_NM', 'day', 'month']
  warnings.warn('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060386 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


/home/workspace/user-workspace/lib/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/workspace/user-workspace/lib/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	train's auc: 0.97497	valid's auc: 0.975907
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	train's auc: 0.97727	valid's auc: 0.977723
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	train's auc: 0.974851	valid's auc: 0.974909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	train's auc: 0.978165	valid's auc: 0.978101
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	train's auc: 0.985248	valid's auc: 0.986293
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	train's auc: 0.983063	valid's auc: 0.984229
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	train's auc: 0.986479	valid's auc: 0.987029
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[8]	t

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[67]	train's auc: 0.989598	valid's auc: 0.990029
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[68]	train's auc: 0.989647	valid's auc: 0.990081
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[69]	train's auc: 0.989718	valid's auc: 0.990142
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[70]	train's auc: 0.989817	valid's auc: 0.990232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[71]	train's auc: 0.989812	valid's auc: 0.990237
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[72]	train's auc: 0.989888	valid's auc: 0.990304
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[73]	train's auc: 0.989871	valid's auc: 0.990295
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[74]	train's auc: 0.98987	valid's auc: 0.990298
[LightGBM

...

[465]	train's auc: 0.99313	valid's auc: 0.992408
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[466]	train's auc: 0.993135	valid's auc: 0.992409
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[467]	train's auc: 0.993139	valid's auc: 0.992411
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[468]	train's auc: 0.993143	valid's auc: 0.992411
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[469]	train's auc: 0.993148	valid's auc: 0.992413
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[470]	train's auc: 0.993154	valid's auc: 0.992416
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[471]	train's auc: 0.993161	valid's auc: 0.99242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[472]	train's auc: 0.993164	valid's auc: 0.992421
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4

[532]	train's auc: 0.993446	valid's auc: 0.992538
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[533]	train's auc: 0.993452	valid's auc: 0.992541
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[534]	train's auc: 0.993454	valid's auc: 0.992543
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[535]	train's auc: 0.993461	valid's auc: 0.992544
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[536]	train's auc: 0.993464	valid's auc: 0.992546
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[537]	train's auc: 0.993472	valid's auc: 0.992549
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[538]	train's auc: 0.993476	valid's auc: 0.992548
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[539]	train's auc: 0.99348	valid's auc: 0.99255
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[599]	train's auc: 0.99374	valid's auc: 0.992642
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[600]	train's auc: 0.993742	valid's auc: 0.992642
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[601]	train's auc: 0.993748	valid's auc: 0.992642
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[602]	train's auc: 0.993751	valid's auc: 0.992643
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[603]	train's auc: 0.993756	valid's auc: 0.992645
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[604]	train's auc: 0.99376	valid's auc: 0.992647
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[605]	train's auc: 0.993764	valid's auc: 0.992649
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[606]	train's auc: 0.993767	valid's auc: 0.99265
[Li

[665]	train's auc: 0.993982	valid's auc: 0.992707
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[666]	train's auc: 0.993986	valid's auc: 0.992708
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[667]	train's auc: 0.993989	valid's auc: 0.992709
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[668]	train's auc: 0.993991	valid's auc: 0.99271
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[669]	train's auc: 0.993996	valid's auc: 0.992712
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[670]	train's auc: 0.994002	valid's auc: 0.992713
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[671]	train's auc: 0.994006	valid's auc: 0.992712
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[672]	train's auc: 0.994011	valid's auc: 0.992714
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[732]	train's auc: 0.994223	valid's auc: 0.99277
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[733]	train's auc: 0.994226	valid's auc: 0.992771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[734]	train's auc: 0.994229	valid's auc: 0.992771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[735]	train's auc: 0.99423	valid's auc: 0.992771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[736]	train's auc: 0.994235	valid's auc: 0.992772


In [ ]:
json.dump(train_results, open(f"{result_dir}{prefix}.result.json", 'w'), indent=2)

# Predict & Submission

In [ ]:
test_df = pd.read_parquet(f"{data_dir}{prefix}_test.parquet")

In [ ]:
submission = pd.read_parquet(test_path)[['TRD_NO']]

In [ ]:
predictions = []

In [ ]:
predictions = []
for i, booster in enumerate(boosters):
    print(f"prediction {i}")
    predictions.append(booster.predict(test_df).tolist())

In [ ]:
for i, predict in enumerate(predictions):
    submission[f"target_{i}"] = predict

In [ ]:
columns = [f"target_{i}" for i in range(len(predictions))]

In [ ]:
columns

In [ ]:
submission['target'] = submission[columns].mean(1)
submission = submission.drop(columns=columns)

In [ ]:
submission.to_feather(f"{submission_dir}prediction.feather")

In [ ]:
submission

In [ ]:
from nipa.taskSubmit import nipa_submit
import os

team_id = "1390"
task_no= "150"
prediction_path = f"{submission_dir}prediction.feather"
# 파일 존재 여부 확인
print("is file: ", os.path.isfile(prediction_path))

# 제출 성공
nipa_submit(team_id=team_id,
            task_no=task_no,
            result=prediction_path
           )
